<a href="https://colab.research.google.com/github/cse583/transformers/blob/main/CreateDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# unzip polybench results zip file
! unzip results-main.zip

Archive:  results-main.zip
ed88af354799f21d7bb14bb8a6082e5e97138847
   creating: results-main/
   creating: results-main/2mm/
  inflating: results-main/2mm/init_array.csv  
  inflating: results-main/2mm/init_array.txt  
  inflating: results-main/2mm/instrumented_2mm  
  inflating: results-main/2mm/kernel_2mm.csv  
  inflating: results-main/2mm/kernel_2mm.txt  
  inflating: results-main/2mm/main.csv  
  inflating: results-main/2mm/main.txt  
  inflating: results-main/2mm/print_array.txt  
  inflating: results-main/2mm/profile.txt  
   creating: results-main/3mm/
  inflating: results-main/3mm/init_array.csv  
  inflating: results-main/3mm/init_array.txt  
  inflating: results-main/3mm/instrumented_3mm  
  inflating: results-main/3mm/kernel_3mm.csv  
  inflating: results-main/3mm/kernel_3mm.txt  
  inflating: results-main/3mm/main.csv  
  inflating: results-main/3mm/main.txt  
  inflating: results-main/3mm/print_array.txt  
  inflating: results-main/3mm/profile.txt  
 extracting: results-

In [3]:
! mv results-main/* .

In [4]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Step 1: Locate and Read All CSV Files
csv_files_train_val = [] # csv files for training and validation
csv_files_test = []      # csv files for testing
base_directory = "."
test_dirs = ["jacobi-2d", "syr2k", "durbin", "2mm"]
# select all files excluding the ones in the test directories
for root, dirs, files in os.walk(base_directory):
  # Modify dirs in-place to exclude specified directories
  dirs[:] = [d for d in dirs if d not in test_dirs]
  for file in files:
    if file.endswith(".csv"):
      csv_files_train_val.append(os.path.join(root, file))
# select the test directory files
for directory in test_dirs:
  # Use glob to find all CSV files in the directory
  files = glob.glob(os.path.join(directory, "*.csv"))
  csv_files_test.extend(files)

print(csv_files_train_val)
print(csv_files_test)

['./adi/init_array.csv', './adi/kernel_adi.csv', './adi/main.csv', './symm/init_array.csv', './symm/main.csv', './symm/kernel_symm.csv', './nussinov/init_array.csv', './nussinov/kernel_nussinov.csv', './nussinov/main.csv', './fdtd-2d/init_array.csv', './fdtd-2d/main.csv', './fdtd-2d/kernel_fdtd_2d.csv', './seidel-2d/kernel_seidel_2d.csv', './seidel-2d/init_array.csv', './seidel-2d/main.csv', './3mm/init_array.csv', './3mm/main.csv', './3mm/kernel_3mm.csv', './mvt/init_array.csv', './mvt/main.csv', './mvt/kernel_mvt.csv', './atax/init_array.csv', './atax/kernel_atax.csv', './atax/main.csv', './bicg/init_array.csv', './bicg/kernel_bicg.csv', './bicg/main.csv', './lu/init_array.csv', './lu/main.csv', './lu/kernel_lu.csv', './gemver/kernel_gemver.csv', './gemver/init_array.csv', './gemver/main.csv', './doitgen/init_array.csv', './doitgen/main.csv', './doitgen/kernel_doitgen.csv', './gesummv/init_array.csv', './gesummv/main.csv', './gesummv/kernel_gesummv.csv', './ludcmp/init_array.csv', '.

In [7]:
# Step 2: Combine All Data
def combine_csv_files(csv_files, dataframes):
  for file in csv_files:
    df = pd.read_csv(file)
    # if len(df.columns) == 2 and df.dtypes[0] == 'object' and pd.api.types.is_integer_dtype(df.iloc[:, 1]):
    if len(df.columns) == 2:
      df = df.iloc[:, :2]  # Select only the first two columns
      df.columns = ["path", "count"]  # Rename columns to standard
      # df["source_file"] = os.path.basename(file)  # Add the file name as a new column
      df["source_file"] = file  # Add the file name as a new column
      dataframes.append(df)

dataframes_train_val = []
data_frames_test = []
combine_csv_files(csv_files_train_val, dataframes_train_val)
combine_csv_files(csv_files_test, data_frames_test)

train_val_df = pd.concat(dataframes_train_val, ignore_index=True)
test_df = pd.concat(data_frames_test, ignore_index=True)

In [32]:
# OPTIONAL: Count the number of unique tokens for each path
# The goal was to see whether BERT is enough to handle the paths since it has max_tokens = 512

# train_val_df["unique_token_count"] = train_val_df["path"].apply(lambda x: len(set(x.split())))
# plt.figure(figsize=(10, 6))
# sns.histplot(train_val_df["unique_token_count"], bins=50, kde=True, color="blue")
# plt.title("Distribution of Unique Token Counts", fontsize=16)
# plt.xlabel("Number of Unique Tokens", fontsize=14)
# plt.ylabel("Frequency", fontsize=14)
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.show()

In [8]:
# Modify the 'count' column where values > 1 become 1, and values <= 1 become 0
train_val_df['label'] = train_val_df['count'].apply(lambda x: 1 if x >= 1 else 0)
test_df['label'] = test_df['count'].apply(lambda x: 1 if x >= 1 else 0)

# Use value_counts to count the number of 1s and 0s
count_summary_train_val = train_val_df['label'].value_counts()
count_summary_test = test_df['label'].value_counts()

In [9]:
# Step 3: Split Dataset
# train_val_df, test_df = train_test_split(combined_df, test_size=0.2, stratify=combined_df["label"], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.15, stratify=train_val_df["label"], random_state=42)

In [10]:
print(train_df)
print(val_df)
print(test_df)

                                                   path   count  \
125   %4 = alloca i32, align 4\n%5 = alloca ptr, ali...       0   
1382  %14 = load i32, ptr %9, align 4\n%15 = icmp sl...       0   
1334  %3 = alloca i32, align 4\n%4 = alloca i32, ali...       0   
892   %3 = alloca i32, align 4\n%4 = alloca i32, ali...       0   
836   %31 = load i32, ptr %15, align 4\n%32 = load i...       0   
...                                                 ...     ...   
910   %14 = load i32, ptr %6, align 4\n%15 = load i3...       0   
11    %7 = alloca i32, align 4\n%8 = alloca i32, ali...       1   
226   %4 = alloca i32, align 4\n%5 = alloca ptr, ali...       0   
51    %101 = load i32, ptr %16, align 4\n%102 = load...  498501   
881   %72 = load i32, ptr %7, align 4\n%73 = load i3...       0   

                         source_file  label  
125   ./nussinov/kernel_nussinov.csv      0  
1382    ./heat-3d/kernel_heat_3d.csv      0  
1334            ./jacobi-1d/main.csv      0  
892        

In [11]:
count_summary_train = train_df['label'].value_counts()
count_summary_val = val_df['label'].value_counts()

print(count_summary_train)
print(count_summary_val)
print(count_summary_test)

label
0    850
1    340
Name: count, dtype: int64
label
0    151
1     60
Name: count, dtype: int64
label
0    92
1    68
Name: count, dtype: int64


In [16]:
# Step 4: Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [14]:
# Set up hugging face
from huggingface_hub import notebook_login
notebook_login()

In [17]:
# Step 5: Save and Upload to Hugging Face
dataset_dict.push_to_hub("zhaojer/compiler_hot_paths")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/zhaojer/compiler_hot_paths/commit/ce33bd342bede348eee9886c0394e6614dadc473', commit_message='Upload dataset', commit_description='', oid='ce33bd342bede348eee9886c0394e6614dadc473', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/zhaojer/compiler_hot_paths', endpoint='https://huggingface.co', repo_type='dataset', repo_id='zhaojer/compiler_hot_paths'), pr_revision=None, pr_num=None)